# **Setup and Dependancies**

In [1]:
import os
import shutil
import random
import yaml
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import kagglehub
from sklearn.model_selection import train_test_split

# Install required packages
!pip install ultralytics

from ultralytics import YOLO
import torch

# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

print("GPU Available:", torch.cuda.is_available())
print("Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.10.19
    Uninstalling nvidia-curand-cu12-10.3

#  ***Actual Dataset Structure***

In [2]:


root_path = "/kaggle/input/bangladeshi-vehicle-license-plate"

print("Inspecting contents of:", root_path)
for root, dirs, files in os.walk(root_path):
    level = root.replace(root_path, '').count(os.sep)
    indent = ' ' * 4 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 4 * (level + 1)
    for f in files[:5]:  # show max 5 files per folder
        print(f"{subindent}{f}")


Inspecting contents of: /kaggle/input/bangladeshi-vehicle-license-plate
bangladeshi-vehicle-license-plate/
    Bangladeshi_Vehicle_Image_with_visible_license_plate/
        Vehicle924.txt
        Vehicle254.txt
        Vehicle1003.txt
        Vehicle75.jpg
        Vehicle1071.jpg


# ***Extract Dataset***

In [3]:
import os
import shutil
import random
from sklearn.model_selection import train_test_split

# Paths
src_dir = "/kaggle/input/bangladeshi-vehicle-license-plate/Bangladeshi_Vehicle_Image_with_visible_license_plate"
project_dir = "/kaggle/working/yolo_license_plate"
dataset_dir = f"{project_dir}/datasets/license_plate"

# Create required YOLO directories
folders = [
    "images/train", "images/val", "images/test",
    "labels/train", "labels/val", "labels/test"
]
for folder in folders:
    os.makedirs(os.path.join(dataset_dir, folder), exist_ok=True)

# List all images
image_files = [f for f in os.listdir(src_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
image_files.sort()

# Step 1: Split into 70% train, 20% test
train_files, test_files = train_test_split(image_files, test_size=0.2, random_state=42)

# Step 2: Split train into 70% train, 30% val
train_files, val_files = train_test_split(train_files, test_size=0.3, random_state=42)

# Dataset dictionary
splits = {
    'train': train_files,
    'val': val_files,
    'test': test_files
}

# Copy files to YOLO directory
for split, files in splits.items():
    for img_file in files:
        # Copy image
        src_img = os.path.join(src_dir, img_file)
        dst_img = os.path.join(dataset_dir, 'images', split, img_file)
        shutil.copy(src_img, dst_img)

        # Copy corresponding label
        label_file = os.path.splitext(img_file)[0] + ".txt"
        src_label = os.path.join(src_dir, label_file)
        dst_label = os.path.join(dataset_dir, 'labels', split, label_file)
        if os.path.exists(src_label):
            shutil.copy(src_label, dst_label)

print("✅ Dataset split (70/30 then 70/30) complete and files copied.")

# Summary
for category in ['images', 'labels']:
    for split in ['train', 'val', 'test']:
        folder = os.path.join(dataset_dir, category, split)
        print(f"{category}/{split}: {len(os.listdir(folder))} files")


✅ Dataset split (70/30 then 70/30) complete and files copied.
images/train: 1080 files
images/val: 463 files
images/test: 386 files
labels/train: 1080 files
labels/val: 463 files
labels/test: 386 files


# ***YAML***

In [4]:
import yaml

data_yaml = {
    'train': '/kaggle/working/yolo_license_plate/datasets/license_plate/images/train',
    'val': '/kaggle/working/yolo_license_plate/datasets/license_plate/images/val',
    'test': '/kaggle/working/yolo_license_plate/datasets/license_plate/images/test',
    'nc': 1,
    'names': ['license_plate']
}

with open('/kaggle/working/yolo_license_plate/data.yaml', 'w') as f:
    yaml.dump(data_yaml, f)

print("✅ data.yaml created.")


✅ data.yaml created.


# ***Training***

In [5]:
from ultralytics import YOLO

# Load YOLOv8 model
model = YOLO('yolov8s.pt')  # You can also use yolov11m.pt or yolov11x.pt

# Train
model.train(
    data='/kaggle/working/yolo_license_plate/data.yaml',
    epochs=50,
    imgsz=640,
    batch=16,
    project='/kaggle/working/yolo_license_plate',
    name='yolov11_license_plate',
    device=0  # or 'cpu'
)


100%|██████████| 21.5M/21.5M [00:00<00:00, 112MB/s] 


Ultralytics 8.3.145 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/yolo_license_plate/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov11_license_plate, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspecti

100%|██████████| 755k/755k [00:00<00:00, 29.6MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 89.0MB/s]


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:04<00:00,  3.30it/s]


                   all        463        472       0.98      0.972      0.988      0.627


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.2ms preprocess, 3.7ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to /kaggle/working/yolo_license_plate/yolov11_license_plate


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x799c01e42750>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

> ***Validate on Test set***

In [6]:
metrics = model.val(
    data='/kaggle/working/yolo_license_plate/data.yaml',
    split='test'
)
print(metrics)


Ultralytics 8.3.145 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1608.0±542.1 MB/s, size: 108.1 KB)


val: Scanning /kaggle/working/yolo_license_plate/datasets/license_plate/labels/test... 386 images, 0 backgrounds, 0 corrupt: 100%|██████████| 386/386 [00:00<00:00, 1431.97it/s]

val: New cache created: /kaggle/working/yolo_license_plate/datasets/license_plate/labels/test.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:05<00:00,  4.68it/s]


                   all        386        394      0.983      0.985      0.993       0.63


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 1.0ms preprocess, 5.4ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to /kaggle/working/yolo_license_plate/yolov11_license_plate2
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x799c0152a150>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.0

# ***Save Zit format***

In [7]:
import zipfile
import os

base_dir = '/kaggle/working/yolo_license_plate/yolov11_license_plate'
zip_path = '/kaggle/working/yolov11_license_plate_full_backup.zip'

with zipfile.ZipFile(zip_path, 'w') as zipf:
    for folder, _, files in os.walk(base_dir):
        for file in files:
            full_path = os.path.join(folder, file)
            arcname = os.path.relpath(full_path, base_dir)
            zipf.write(full_path, arcname=arcname)

print("✅ Full model folder zipped as:", zip_path)


✅ Full model folder zipped as: /kaggle/working/yolov11_license_plate_full_backup.zip
